In [31]:
import pandas as pd
import numpy as np

In [32]:
base = pd.read_csv('C:/Users/hbati/Ironhack/Cases/AME/base_de_conhecimento.csv')

In [33]:
base #describes each of the 129 questions asked

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,Hobby,Do you code as a hobby?
2,OpenSource,Do you contribute to open source projects?
3,Country,In which country do you currently reside?
4,Student,"Are you currently enrolled in a formal, degree..."
...,...,...
124,Age,"What is your age? If you prefer not to answer,..."
125,Dependents,Do you have any children or other dependents t...
126,MilitaryUS,Are you currently serving or have you ever ser...
127,SurveyTooLong,How do you feel about the length of the survey...


In [34]:
data_ori = pd.read_csv('C:/Users/hbati/Ironhack/Cases/AME/survey_results_public.csv',low_memory=False)

In [4]:
data = pd.read_csv('C:/Users/hbati/Ironhack/Cases/AME/survey_results_public.csv',low_memory=False)

<IPython.core.display.Javascript object>

In [144]:
data.tail() #98855 entries

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
98850,101513,Yes,Yes,United States,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98851,101531,No,Yes,Spain,"Yes, full-time","Not employed, but looking for work",NaN,NaN,NaN,Back-end developer;Front-end developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98852,101541,Yes,Yes,India,"Yes, full-time",Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98853,101544,Yes,No,Russian Federation,No,"Independent contractor, freelancer, or self-em...",Some college/university study without earning ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98854,101548,Yes,Yes,Cambodia,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
data.shape

(98855, 129)

<!--
• Salário vazio ou com valor “NA” deve ser convertido para zero (0.0).
    # done
• Salário deve ser sempre calculado em reais e mensal. Para esse cálculo você usará a coluna
ConvertedSalary, que contém o salário anual. Considere que 1 dólar equivale a R$3,81.
    # done, but with updated exchange rate of R$5,15
• O nome dos respondentes deve seguir a regra respondente_[número] . (ex: respondente_1,
respondente_2, respondente_3). O critério de geração desse número é todo seu.
• Cada linha da tabela linguagem_programacao deve conter uma única linguagem de
programação.
• Cada linha da tabela ferramenta_comunic deve conter apenas uma ferramenta de
comunicação.
-->

In [22]:
data[['Currency','Salary','SalaryType','ConvertedSalary']].isna().sum() # convert nans in Salary and Converted Salary to 0.0

# about half of salary information is lost

Currency           36847
Salary             48277
SalaryType         47785
ConvertedSalary    51153
dtype: int64

In [23]:
data['Salary'] = data['Salary'].fillna(0)
data['ConvertedSalary'] = data['ConvertedSalary'].fillna(0)

In [24]:
data[['Currency','Salary','SalaryType','ConvertedSalary']].isna().sum() # done

Currency           36847
Salary                 0
SalaryType         47785
ConvertedSalary        0
dtype: int64

In [25]:
data['Salary'].dtypes # object, but ConvertedSalary already has commas converted to dots, and it should be all floats

dtype('O')

In [26]:
data['ConvertedSalary'].dtypes # float, as expected

dtype('float64')

In [27]:
#ConvertedSalary: Salary converted to annual USD salaries using the exchange rate on 2018-01-18, assuming 12 working months and 50 working weeks

In [28]:
data[data['Salary'].str.contains(',') == True]['Salary'] # There might be some confusion between ',' and '.' depending on currency? Let's explore..

833         2688,82
1449       35000,00
2365      2,000,000
2506        7080,65
2939     112,000.00
            ...    
87785     35,000.00
87841    500,000.00
87868        455,00
89246      1,500.00
89374      3,000.00
Name: Salary, Length: 251, dtype: object

In [29]:
data['Currency'].value_counts()

U.S. dollars ($)               20601
Euros (€)                      15201
Indian rupees (₹)               7908
British pounds sterling (£)     4856
Canadian dollars (C$)           2535
Russian rubles (₽)              1768
Brazilian reais (R$)            1663
Australian dollars (A$)         1571
Polish złoty (zł)               1434
Swedish kroner (SEK)             864
Swiss francs                     671
Chinese yuan renminbi (¥)        512
Danish krone (kr)                461
Mexican pesos (MXN$)             457
South African rands (R)          432
Norwegian krone (kr)             423
Singapore dollars (S$)           263
Japanese yen (¥)                 241
Bitcoin (btc)                    147
Name: Currency, dtype: int64

In [30]:
data[['Currency','Salary','SalaryType','ConvertedSalary']].iloc[87841] #how is this conversion made if currency is NaN?

Currency                  NaN
Salary             500,000.00
SalaryType            Monthly
ConvertedSalary         444.0
Name: 87841, dtype: object

In [31]:
data[['Currency','Salary','SalaryType','ConvertedSalary']].iloc[87868] #checks out, monthly euros to yearly dollars

Currency           Euros (€)
Salary                455,00
SalaryType           Monthly
ConvertedSalary     668232.0
Name: 87868, dtype: object

In [32]:
# bottomline, let's stick with ConvertedSalary which standardizes Salary.

In [33]:
data['SalaryType'].value_counts()

Monthly    26252
Yearly     22556
Weekly      2262
Name: SalaryType, dtype: int64

In [34]:
data[['Currency','Salary','SalaryType','ConvertedSalary']].head(20)
#ConvertedSalary: Salary converted to annual USD salaries using the exchange rate on 2018-01-18, assuming 12 working months and 50 working weeks

,Currency,Salary,SalaryType,ConvertedSalary
0,NaN,0,Monthly,0.0
1,British pounds sterling (£),51000,Yearly,70841.0
2,NaN,0,NaN,0.0
3,U.S. dollars ($),0,NaN,0.0
4,South African rands (R),260000,Yearly,21426.0
5,British pounds sterling (£),30000,NaN,41671.0
6,U.S. dollars ($),120000,Yearly,120000.0
7,NaN,0,NaN,0.0
8,U.S. dollars ($),250000,Yearly,250000.0
9,NaN,0,NaN,0.0


In [35]:
#let's add a column where ConvertedSalary is converted to monthly reais

In [49]:
#'CSMR' stands for ConvertedSalaryMonthlyReais
data['CSMR'] = (data['ConvertedSalary']/12)*3.81 #as of 10/04/2022

In [50]:
data[['Currency','Salary','SalaryType','ConvertedSalary','CSMR']].head(30)

,Currency,Salary,SalaryType,ConvertedSalary,CSMR
0,NaN,0,Monthly,0.0,0.0000
1,British pounds sterling (£),51000,Yearly,70841.0,22492.0175
2,NaN,0,NaN,0.0,0.0000
3,U.S. dollars ($),0,NaN,0.0,0.0000
4,South African rands (R),260000,Yearly,21426.0,6802.7550
5,British pounds sterling (£),30000,NaN,41671.0,13230.5425
6,U.S. dollars ($),120000,Yearly,120000.0,38100.0000
7,NaN,0,NaN,0.0,0.0000
8,U.S. dollars ($),250000,Yearly,250000.0,79375.0000
9,NaN,0,NaN,0.0,0.0000


In [51]:
# ConvertedSalary might be standardized, but it's definitely not clean, e.g.
data.iloc[22][['Currency','Salary','SalaryType','ConvertedSalary','CSMR']]

Currency           British pounds sterling (£)
Salary                                      25
SalaryType                             Monthly
ConvertedSalary                          420.0
CSMR                                    133.35
Name: 22, dtype: object

In [163]:
# The conversion to yearly dollars checks out, but the salary value itself has to be wrong. We might want to establish mininum values.
# This should become clearer once we plot the data.

In [164]:
#Convert column 'Respondent' (with randomized IDs) to sequential Respondent_1, Respondent_2 and so forth

In [41]:
data[['Respondent']].reset_index()

,index,Respondent
0,0,1
1,1,3
2,2,4
3,3,5
4,4,7
...,...,...
98850,98850,101513
98851,98851,101531
98852,98852,101541
98853,98853,101544


In [42]:
data['Respondent'] = 'Respondent_' +(data['Respondent'].reset_index()['index'] + 1).astype(str)


In [44]:
data.loc[data['Respondent']=='Respondent_323'][['Respondent','Currency','Salary','SalaryType','ConvertedSalary','CSMR']]

,Respondent,Currency,Salary,SalaryType,ConvertedSalary,CSMR
322,Respondent_323,Brazilian reais (R$),50000,Monthly,187152.0,80319.4


In [167]:
data.tail()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy,CSMR
98850,Respondent_98851,Yes,Yes,United States,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
98851,Respondent_98852,No,Yes,Spain,"Yes, full-time","Not employed, but looking for work",NaN,NaN,NaN,Back-end developer;Front-end developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
98852,Respondent_98853,Yes,Yes,India,"Yes, full-time",Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
98853,Respondent_98854,Yes,No,Russian Federation,No,"Independent contractor, freelancer, or self-em...",Some college/university study without earning ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
98854,Respondent_98855,Yes,Yes,Cambodia,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [168]:
# Lets change Yes and No to 1 and 0 respectively
data['OpenSource'] = data['OpenSource'].replace({'No': 0, 'Yes': 1})
data['Hobby'] = data['Hobby'].replace({'No': 0, 'Yes': 1})

In [215]:
#new column for Student
data['StudentYN'] = data['Student']

In [216]:
data['StudentYN'] = data['StudentYN'].replace({'No': 0, 'Yes, full-time': 1})

In [217]:
data['StudentYN'] = data['StudentYN'].replace({'Yes, part-time': 1})

In [218]:
data['Student'].value_counts()

No                70399
Yes, full-time    18394
Yes, part-time     6108
Name: Student, dtype: int64

In [214]:
data['StudentYN'].value_counts() #done

0.0    70399
1.0    24502
Name: StudentYN, dtype: int64

In [169]:
data.head()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy,CSMR
0,Respondent_1,1,0,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,...,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy,0.000000
1,Respondent_2,1,1,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,...,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,30402.595833
2,Respondent_3,1,1,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,Engineering manager;Full-stack developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,Respondent_4,0,0,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,Full-stack developer,...,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,35 - 44 years old,No,No,The survey was an appropriate length,Somewhat easy,0.000000
4,Respondent_5,1,0,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,...,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,9195.325000


Now organizing the SQL tables

In [65]:
#linguagem_programacao:
data['LanguageWorkedWith']

0                              JavaScript;Python;HTML;CSS
1                            JavaScript;Python;Bash/Shell
2                                                     NaN
3        C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell
4                      C;C++;Java;Matlab;R;SQL;Bash/Shell
                               ...                       
98850                                                 NaN
98851                                                 NaN
98852                                                 NaN
98853                                                 NaN
98854                                                 NaN
Name: LanguageWorkedWith, Length: 98855, dtype: object

In [66]:
data['LanguageWorkedWith'] = data['LanguageWorkedWith'].str.split(';')

In [67]:
data['LanguageWorkedWith']

0                          [JavaScript, Python, HTML, CSS]
1                         [JavaScript, Python, Bash/Shell]
2                                                      NaN
3        [C#, JavaScript, SQL, TypeScript, HTML, CSS, B...
4               [C, C++, Java, Matlab, R, SQL, Bash/Shell]
                               ...                        
98850                                                  NaN
98851                                                  NaN
98852                                                  NaN
98853                                                  NaN
98854                                                  NaN
Name: LanguageWorkedWith, Length: 98855, dtype: object

In [68]:
languages = data[['Respondent','LanguageWorkedWith']].explode('LanguageWorkedWith')

In [69]:
languages

,Respondent,LanguageWorkedWith
0,Respondent_1,JavaScript
0,Respondent_1,Python
0,Respondent_1,HTML
0,Respondent_1,CSS
1,Respondent_2,JavaScript
...,...,...
98850,Respondent_98851,NaN
98851,Respondent_98852,NaN
98852,Respondent_98853,NaN
98853,Respondent_98854,NaN


In [70]:
#ferramentas_programacao:
data_ori['CommunicationTools'][5]

'Confluence;Jira;Office / productivity suite (Microsoft Office, Google Suite, etc.);Other chat system (IRC, proprietary software, etc.)'

In [71]:
data['CommunicationTools'] = data_ori['CommunicationTools'].str.split(';')

In [72]:
data['CommunicationTools'][1]

['Confluence',
 'Office / productivity suite (Microsoft Office, Google Suite, etc.)',
 'Slack',
 'Other wiki tool (Github, Google Sites, proprietary software, etc.)']

In [73]:
tools = data[['Respondent','CommunicationTools']].explode('CommunicationTools')

In [74]:
tools

,Respondent,CommunicationTools
0,Respondent_1,Slack
1,Respondent_2,Confluence
1,Respondent_2,"Office / productivity suite (Microsoft Office,..."
1,Respondent_2,Slack
1,Respondent_2,"Other wiki tool (Github, Google Sites, proprie..."
...,...,...
98850,Respondent_98851,NaN
98851,Respondent_98852,NaN
98852,Respondent_98853,NaN
98853,Respondent_98854,NaN


Plotting

In [ ]:
data.head()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy,CSMR
0,Respondent_1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,...,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy,0.000000
1,Respondent_2,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,...,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,30402.595833
2,Respondent_3,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,Engineering manager;Full-stack developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,Respondent_4,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,Full-stack developer,...,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,35 - 44 years old,No,No,The survey was an appropriate length,Somewhat easy,0.000000
4,Respondent_5,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,...,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,9195.325000


we need to create some new tables for SQL to ingest:

In [125]:
#Countries table

Country = pd.DataFrame(data['Country'].unique(),columns=['Country'])
Country

#data['Country'].isna().sum() yields 412 nulls

,Country
0,Kenya
1,United Kingdom
2,United States
3,South Africa
4,Nigeria
...,...
179,Eritrea
180,Democratic People's Republic of Korea
181,Djibouti
182,NaN


In [126]:
Country.sort_values(by='Country',inplace=True)
Country

,Country
78,Afghanistan
85,Albania
27,Algeria
150,Andorra
108,Angola
...,...
45,Viet Nam
100,Yemen
160,Zambia
119,Zimbabwe


In [127]:
Country.reset_index(drop=True,inplace=True)
Country

,Country
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
179,Viet Nam
180,Yemen
181,Zambia
182,Zimbabwe


In [128]:
Country.reset_index(inplace=True)

In [129]:
Country

,index,Country
0,0,Afghanistan
1,1,Albania
2,2,Algeria
3,3,Andorra
4,4,Angola
...,...,...
179,179,Viet Nam
180,180,Yemen
181,181,Zambia
182,182,Zimbabwe


In [130]:
Country = Country.rename(columns = {'index':'Country_id'}) #Done!
Country

,Country_id,Country
0,0,Afghanistan
1,1,Albania
2,2,Algeria
3,3,Andorra
4,4,Angola
...,...,...
179,179,Viet Nam
180,180,Yemen
181,181,Zambia
182,182,Zimbabwe


In [117]:
#Companies table
Companies= pd.DataFrame(data['CompanySize'].unique(),columns=['CompanySize'])
Companies

# data['CompanySize'].isna().sum() yields 27324 values.. I'll keep them

,CompanySize
0,20 to 99 employees
1,"10,000 or more employees"
2,100 to 499 employees
3,10 to 19 employees
4,500 to 999 employees
5,"1,000 to 4,999 employees"
6,"5,000 to 9,999 employees"
7,Fewer than 10 employees
8,NaN


In [118]:
Companies = Companies.reindex([7,3,0,2,4,5,6,1,8])
Companies 

,CompanySize
7,Fewer than 10 employees
3,10 to 19 employees
0,20 to 99 employees
2,100 to 499 employees
4,500 to 999 employees
5,"1,000 to 4,999 employees"
6,"5,000 to 9,999 employees"
1,"10,000 or more employees"
8,NaN


In [119]:
Companies = Companies.reset_index(drop=True)
Companies

,CompanySize
0,Fewer than 10 employees
1,10 to 19 employees
2,20 to 99 employees
3,100 to 499 employees
4,500 to 999 employees
5,"1,000 to 4,999 employees"
6,"5,000 to 9,999 employees"
7,"10,000 or more employees"
8,NaN


In [120]:
Companies = Companies.reset_index()
Companies

,index,CompanySize
0,0,Fewer than 10 employees
1,1,10 to 19 employees
2,2,20 to 99 employees
3,3,100 to 499 employees
4,4,500 to 999 employees
5,5,"1,000 to 4,999 employees"
6,6,"5,000 to 9,999 employees"
7,7,"10,000 or more employees"
8,8,NaN


In [121]:
Companies = Companies.rename(columns = {'index':'Company_id'}) #Done!
Companies

,Company_id,CompanySize
0,0,Fewer than 10 employees
1,1,10 to 19 employees
2,2,20 to 99 employees
3,3,100 to 499 employees
4,4,500 to 999 employees
5,5,"1,000 to 4,999 employees"
6,6,"5,000 to 9,999 employees"
7,7,"10,000 or more employees"
8,8,NaN


In [86]:
# Now for OperatingSystem

OS = pd.DataFrame(data['OperatingSystem'].unique(),columns=['OperatingSystem'])
OS

# data['OperatingSystem'].isna().sum() yields 22676 nulls.. maybe I will keep it

,OperatingSystem
0,Linux-based
1,NaN
2,Windows
3,MacOS
4,BSD/Unix


In [87]:
OS = OS.reindex([2,3,0,4,1])
OS

,OperatingSystem
2,Windows
3,MacOS
0,Linux-based
4,BSD/Unix
1,NaN


In [88]:
OS = OS.reset_index(drop=True)
OS

,OperatingSystem
0,Windows
1,MacOS
2,Linux-based
3,BSD/Unix
4,NaN


In [89]:
OS = OS.reset_index().rename(columns = {'index':'OS_id'}) #Done!
OS

,OS_id,OperatingSystem
0,0,Windows
1,1,MacOS
2,2,Linux-based
3,3,BSD/Unix
4,4,NaN


In [90]:
OS

,OS_id,OperatingSystem
0,0,Windows
1,1,MacOS
2,2,Linux-based
3,3,BSD/Unix
4,4,NaN


In [91]:
# Now for Resp_use_language
language = pd.DataFrame(languages['LanguageWorkedWith'].unique(),columns=['LanguageWorkedWith'])
language.head(10) #puts como que eu faço pra jogar o nan pro final?

,LanguageWorkedWith
0,JavaScript
1,Python
2,HTML
3,CSS
4,Bash/Shell
5,NaN
6,C#
7,SQL
8,TypeScript
9,C


In [92]:
language = language.reset_index()
language = language.rename(columns = {'index':'Language_id'})

language # done! Now to create a table with respondent and language_id

,Language_id,LanguageWorkedWith
0,0,JavaScript
1,1,Python
2,2,HTML
3,3,CSS
4,4,Bash/Shell
5,5,NaN
6,6,C#
7,7,SQL
8,8,TypeScript
9,9,C


In [93]:
languages

,Respondent,LanguageWorkedWith
0,Respondent_1,JavaScript
0,Respondent_1,Python
0,Respondent_1,HTML
0,Respondent_1,CSS
1,Respondent_2,JavaScript
...,...,...
98850,Respondent_98851,NaN
98851,Respondent_98852,NaN
98852,Respondent_98853,NaN
98853,Respondent_98854,NaN


In [94]:
# Now for Resp_usa_linguagem
resp_usa_linguagem = pd.merge(left=languages,right=language,how='left').drop(columns='LanguageWorkedWith')

In [95]:
resp_usa_linguagem #done!

,Respondent,Language_id
0,Respondent_1,0
1,Respondent_1,1
2,Respondent_1,2
3,Respondent_1,3
4,Respondent_2,0
...,...,...
506184,Respondent_98851,5
506185,Respondent_98852,5
506186,Respondent_98853,5
506187,Respondent_98854,5


In [96]:
# Now for Resp_usa_ferramenta
tools

,Respondent,CommunicationTools
0,Respondent_1,Slack
1,Respondent_2,Confluence
1,Respondent_2,"Office / productivity suite (Microsoft Office,..."
1,Respondent_2,Slack
1,Respondent_2,"Other wiki tool (Github, Google Sites, proprie..."
...,...,...
98850,Respondent_98851,NaN
98851,Respondent_98852,NaN
98852,Respondent_98853,NaN
98853,Respondent_98854,NaN


In [97]:
tool = pd.DataFrame(tools['CommunicationTools'].unique(),columns=['CommunicationTools'])

In [98]:
tool = tool.reset_index()
tool = tool.rename(columns = {'index':'Tool_id'})


In [99]:
tool #Done!

,Tool_id,CommunicationTools
0,0,Slack
1,1,Confluence
2,2,"Office / productivity suite (Microsoft Office,..."
3,3,"Other wiki tool (Github, Google Sites, proprie..."
4,4,NaN
5,5,Jira
6,6,"Other chat system (IRC, proprietary software, ..."
7,7,Stack Overflow Enterprise
8,8,Facebook
9,9,Google Hangouts/Chat


In [100]:
tools

,Respondent,CommunicationTools
0,Respondent_1,Slack
1,Respondent_2,Confluence
1,Respondent_2,"Office / productivity suite (Microsoft Office,..."
1,Respondent_2,Slack
1,Respondent_2,"Other wiki tool (Github, Google Sites, proprie..."
...,...,...
98850,Respondent_98851,NaN
98851,Respondent_98852,NaN
98852,Respondent_98853,NaN
98853,Respondent_98854,NaN


In [101]:
resp_usa_ferramenta = pd.merge(left=tools,right=tool,how='left').drop(columns='CommunicationTools')

In [102]:
resp_usa_ferramenta

,Respondent,Tool_id
0,Respondent_1,0
1,Respondent_2,1
2,Respondent_2,2
3,Respondent_2,0
4,Respondent_2,3
...,...,...
198208,Respondent_98851,4
198209,Respondent_98852,4
198210,Respondent_98853,4
198211,Respondent_98854,4


In [103]:
# Now for ferramenta_comunic
ferramenta_comunic = tool

In [104]:
# Now for linguagem_programacao
linguagem_programacao = language

In [ ]:
# Now for Respondent

In [53]:
Respondent_pre = data[['Respondent','OpenSource','Hobby','CSMR','ConvertedSalary','OperatingSystem','Country','CompanySize']]
Respondent_pre

,Respondent,OpenSource,Hobby,CSMR,ConvertedSalary,OperatingSystem,Country,CompanySize
0,Respondent_1,No,Yes,0.0000,0.0,Linux-based,Kenya,20 to 99 employees
1,Respondent_2,Yes,Yes,22492.0175,70841.0,Linux-based,United Kingdom,"10,000 or more employees"
2,Respondent_3,Yes,Yes,0.0000,0.0,NaN,United States,20 to 99 employees
3,Respondent_4,No,No,0.0000,0.0,Windows,United States,100 to 499 employees
4,Respondent_5,No,Yes,6802.7550,21426.0,Windows,South Africa,"10,000 or more employees"
...,...,...,...,...,...,...,...,...
98850,Respondent_98851,Yes,Yes,0.0000,0.0,NaN,United States,NaN
98851,Respondent_98852,Yes,No,0.0000,0.0,NaN,Spain,NaN
98852,Respondent_98853,Yes,Yes,0.0000,0.0,NaN,India,NaN
98853,Respondent_98854,No,Yes,0.0000,0.0,NaN,Russian Federation,NaN


In [54]:
Respondente = pd.merge(left=Respondent_pre,right=OS,how='left') #merged OS
Respondente = pd.merge(left=Respondente,right=Companies,how='left') #merged Companies
Respondente = pd.merge(left=Respondente,right=Country,how='left') #merged Country
Respondente = Respondente.drop(columns=['OperatingSystem','Country','CompanySize'])

<IPython.core.display.Javascript object>

NameError: name 'OS' is not defined

Organizing all the tables I need to transfer to MySQL

In [193]:
#sistema_operacional:
OS
#país:
Country
#empresa:
Companies
#linguagem:
language
#ferramenta_comunic:
tool
#resp_usa_ferramenta:
resp_usa_ferramenta
#resp_usa_linguagem:
resp_usa_linguagem
#respondente:
Respondente

,Respondent,OpenSource,Hobby,CSMR,OS_id,Company_id,Country_id
0,Respondent_1,0,1,0.000000,2,2,85
1,Respondent_2,1,1,30402.595833,2,7,173
2,Respondent_3,1,1,0.000000,4,2,175
3,Respondent_4,0,0,0.000000,0,3,175
4,Respondent_5,0,1,9195.325000,0,7,150
...,...,...,...,...,...,...,...
98850,Respondent_98851,1,1,0.000000,4,8,175
98851,Respondent_98852,1,0,0.000000,4,8,152
98852,Respondent_98853,1,1,0.000000,4,8,74
98853,Respondent_98854,0,1,0.000000,4,8,137


In [232]:
OS.to_csv('OS.csv')

In [233]:
Country.to_csv('Country.csv')

In [234]:
Companies.to_csv('Companies.csv')

In [235]:
language.to_csv('language.csv')

In [236]:
tool.to_csv('tool.csv')

In [237]:
resp_usa_ferramenta.to_csv('resp_usa_ferramenta.csv')

In [238]:
resp_usa_linguagem.to_csv('resp_usa_linguagem.csv')

In [239]:
Respondente.to_csv('Respondente')

In [241]:
import sqlalchemy as db

db_server='pymysql'
user='root'
db_port = '3306'
password = 'password'
ip = '127.0.0.1'
db_name = 'ame' # schema do sql check
engine = db.create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')
conn = engine.connect()


In [245]:
OS.to_sql('os', conn, if_exists='replace', index=False)


5

In [246]:
Country.to_sql('country', conn, if_exists='replace', index=False)

184

In [247]:
Companies.to_sql('companies', conn, if_exists='replace', index=False)

9

In [248]:
language.to_sql('language', conn, if_exists='replace', index=False)

39

In [249]:
tool.to_sql('tool', conn, if_exists='replace', index=False)

12

In [251]:
resp_usa_ferramenta.to_sql('resp_usa_ferramenta', conn, if_exists='replace', index=False)

198213

In [252]:
resp_usa_linguagem.to_sql('resp_usa_linguagem', conn, if_exists='replace', index=False)

506189

In [253]:
Respondente.to_sql('respondente', conn, if_exists='replace', index=False)

98855